In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Berechnung der Endspieldatenbank

Wie in TODO: Kapitel bereits erklärt ist ein Schachspiel gewonnen, wenn die gegnerische Figur mattgesetzt wurde. 

Bei einer geringen Anzahl an Figuren $P$ im Spielzustand lassen sich alle möglichen Positionen berechnen.
Aus diesen kann eine Strategie entwickelt werden, den Gegner zu schlagen.

Folgend werden folgende Definitionen verwendet:
* $board.pieces$: Liste der Figuren, welche in einem Zustand vorhanden sind.
* $validBoards$: Alle Zustände des Schachspiels, die gegen keine Regeln verstoßen.
* $wonBoards$: Alle Zustände des Schachspiels, in denen ein Spieler gewonnen hat.
* $previousStates(b)$: Alle Zustände, aus denen durch Ausführen eines einzelnen Zuges der Zustand $b$ erreicht werden kann.

Seien alle möglichen (validen) Kombinationen von Positionen der Figuren $P$ die Menge $S$.  
Für $S$ gilt:  
$$
board \in S \implies \forall p \in P : p \in board.pieces \\
\land \\
board \in S \implies board \in validBoards
$$

Aus der Menge $S$ lassen sich Zustände auswählen, welche $n$ Züge vom Sieg entfernt sind. 
Diese Zustände lassen sich in $S_n$ zusammenfassen. Ist das Spiel gewonnen, verbleiben 0 Züge bis zum Sieg.  
Für alle diese Zustände, in denen ein Spieler mattgesetzt ist, gilt:  
$$board \in S_0 \implies board \in wonBoards \land board \in S$$  
Aus dieser Definition können induktiv die verbleibenden $S_n$ hergeleitet werden:  
$$board \in S_{n+1} \iff board \in S \land \exists b \in S_n: board \in previousStates(b)$$ 

Für die Berechnungen in diesem Notebook gilt da für den schwarzen Spieler immer nur der König auf dem Feld steht weiter Folgendes:
* $board.turn$: Der Spieler, welcher am Zug ist

$$
n \equiv 0 \mod 2 \implies \forall b \in S_n : b.turn = schwarz \\
\land \\
n \equiv 1 \mod 2 \implies \forall b \in S_n : b.turn = weiß \\
$$

Dieses Notebook wird zur Berechnung der $S_n$ Mengen verwendet. Diese werden benötigt, um letztendlich ein Schach-Endspiel lösen zu können. 

## Ein Hinweis zu Spiegelungen
In diesem Notebook werden Spiegelungen der Situationen verwendet. Die technische Umsetzung 
dieser Spiegelungen werden im Verlauf des Dokuments erklärt, an dieser Stelle soll lediglich
eine Einführung in die Theorie hinter dem Spiegeln von Situationen erklärt werden.

Durch die zuvor erklärten Bewegungsmuster der Figuren sind Schachbretter in vielen Fällen symmetrisch.

Eine Position mit dem Turm in "a8", der Dame in "g6" und dem gegnerischen König in "h8" ist genauso verloren wie 
dieselbe Position nur mit dem Turm in "a1", der Dame in "g3" und dem König in "h1".
Dies wäre eine Spiegelung entlang der horizontalen zwischen den Zeilen 4 und 5.
Weiter sind auch Spiegelungen entlang der vertikalen (Zwischen Reihe e und f), den Diagonalen und Rotationen 
(jeweils um 90°, 180° und 270°) möglich.

Durch das simple Spiegeln der Spielsituationen, können aus einer validen Spielsituation bis zu sieben weitere ohne
großen Rechenaufwand erstellen. Aus diesem Grund werden in diesem Dokument bei jeder Berechnung neuer Situationen diese
gespiegelt und die Spiegelungen ebenfalls überprüft und abgespeichert. 

Spiegelungen entlang der Diagonalen erfordern eine Überprüfung, wo sich diagonal bewegende Figuren (Dame, Läufer, Springer) auf dem Spielfeld befinden.
Da diese Überprüfung den Geschwindigkeitsvorteil der Spiegelung negiert, werden sie nicht durchgeführt.

Da Bauern sich nur in eine Richtung bewegen können, lassen sich nur Spielsituationen spiegeln, welche keine Bauern enthalten.

## Ein Hinweis zur effizienten Ergebnisverwaltung
Im Verlauf der Berechnung muss mehrfach überprüft werden, ob eine Situation bereits bekannt und einem $S_n$ zugeordnet ist.
Da der Abgleich mit einer Liste in Python ineffizient ist, findet dieser Abgleich mit Mengen statt.
Mengen werden in Python als Hash-Tabellen umgesetzt und haben damit eine Zeitkomplexität bei der Überprüfung, ob sie ein bestimmtes
Element enthalten von $\mathcal{O}(1)$.
`board` Objekte der `chess` Library unterstützen jedoch nicht 


## Funktionen zur Bestimmung aller gültigen Positionen

Ein Schachbrett besteht aus insgesamt acht Spalten und Zeilen. Die Spalten werden durch Buchstaben gekennzeichnet, 
die Zeilen durch Zahlen. Aus der Kombination einer Spalte (z.B. a) und einer Zahl (z.B. 1) erhält man eine 
eindeutige Kennzeichnung für ein Feld (z.B. a1).  
Die folgende Funktion kombiniert die Buchstaben a bis h mit den Zahlen 1 bis 8 zu Feldnamen und gibt diese zurück.

In [ ]:
def get_all_squares():
    columns = {
        1 : 'a',
        2 : 'b',
        3 : 'c',
        4 : 'd',
        5 : 'e',
        6 : 'f',
        7 : 'g',
        8 : 'h'
    }
    
    all_squares = []
    for row in range(1,9):
        for col_num in range(1,9):
            column = columns[col_num]
            all_squares.append(chess.parse_square(column + str(row)))
            
    return all_squares

Das Erstellen jeglicher Boards wird mit der Funktion ``place_piece_everywhere_on_every_board`` umgesetzt. 
Diese erhält folgende Parameter: 

* ``piece``: Die zu platzierende Figur als Objekt der chess-Library
* ``list_of_boards``: Eine Liste mit Board-Objekten, auf welchen die Figur platziert werden soll

Die Funktion betrachtet jede Situation in der `list_of_boards`. Das übergebene `piece` wird auf jeden freien 
Platz dieser Situation platziert. Jedes Mal, wenn eine Figur platziert wird, wird eine Kopie des Board-Objektes erstellt, 
die ``list_of_boards`` wird folglich nicht verändert.  
Wenn der zweite König platziert wird, wird die Situation zusätzlich auf Validität überprüft.
Wenn alle Figuren platziert wurden, werden nur Boards, in denen Schwarz matt ist, zurückgegeben. 

Um die Effizienz der Berechnung zu erhöhen werden die Schachbretter gespiegelt.
Damit bei folgenden Berechnungen Situationen nicht einmal durch Spiegelung und einmal durch
Bewegung von Figuren erreicht werden, werden die ungespiegelten Situationen in der Liste `uniques`
gespeichert.

Die Funktion gibt als Ergebnis eine Liste aller generierten Zustände als ``result_list``, 
die Menge der bereits verwendeten Situationen `used_boards` und alle ungespiegelten-Boards (vor Spiegelungen) `uniques` zurück.

In [ ]:
def place_figure_everywhere_on_every_board(piece, list_of_boards, piece_count, user_wants_pawn):
    result_list = []
    uniques = []
    all_squares = get_all_squares()
    used_boards = set()
    
    for board in list_of_boards:
        squares_used = list(board.piece_map().keys())
        for square in all_squares:
            if square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(square, piece)
                
                if len(squares_used) >= 1 and not tmp_board.is_valid(): 
                    # Don't process invalid boards further 
                    # than the second king
                    continue
                    
                outcome = tmp_board.outcome()
                if outcome is not None:
                    rep = (tmp_board.turn, tmp_board.__str__())
                    if outcome.winner is not None \
                            and tmp_board.is_valid() \
                            and rep not in used_boards:
                        
                        uniques.append(tmp_board)
                        result_list.append(tmp_board)
                        used_boards.add((tmp_board.turn,tmp_board.__str__()))
                        if not user_wants_pawn:
                            for swt in Swap_Type:
                                mir_board = mirror(tmp_board, swt)
                                if (mir_board.turn, mir_board.__str__()) not in used_boards:
                                    result_list.append(mir_board)
                                    used_boards.add((mir_board.turn,mir_board.__str__()))
                        continue
                        
                if len(squares_used) + 1 < piece_count: #Board is valid, but needs more pieces
                    result_list.append(tmp_board)
    return result_list, used_boards, uniques

## Die Ursprungsmenge $S_0$ erstellen
Als Basis der Berechnung dient die Liste $S_0$. Diese enthält alle möglichen Konstellationen der Spielfiguren auf dem
Spielbrett, in denen Weiß Schwarz besiegt hat.
Hierfür werden die Figuren mit der Funktion ``place_figure_everywhere_on_every_board`` auf allen 
Positionen platziert.   

Die Funktion `setup_boards` automatisiert dies und gibt die Liste $S_0$, eine Menge der bereits bekannten Situationen 
`used_boards` und die Menge der ungespiegelten Situationen `uniques` zurück.

In [ ]:
def setup_boards(user_supplied_pieces, user_wants_pawn):
    pieces_to_place = create_piece_list(user_supplied_pieces)
    
    empty_board = chess.Board().empty()
    empty_board.turn = chess.BLACK
    s_0 = [empty_board]
    
    piece_count = len(pieces_to_place)
    for piece in pieces_to_place:
        s_0, used_boards, uniques = place_figure_everywhere_on_every_board(piece, s_0, piece_count, user_wants_pawn)

    print(str(len(s_0)) + " Boards in S_0")
    return s_0, used_boards, uniques

# A queen will automatically be replaced by a pawn
def create_piece_list(user_supplied_pieces):
    if chess.Piece.from_symbol("P") in user_supplied_pieces:
        user_supplied_pieces.remove(chess.Piece.from_symbol("P"))
        user_supplied_pieces.append(chess.Piece.from_symbol("Q"))
        
    return [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k")] \
           + user_supplied_pieces

## Rückwärts neue Situationen bestimmen

Der nächste Schritt besteht darin, sämtliche $S_{n}$ Mengen zu bestimmen. Hierzu wird eine bereits bestimmte $S_n$
Menge genommen und alle Situationen berechnet, die durch Durchführen eines Zugs zu einer Situation aus $S_n$ werden.

Besonders muss bei dieser Art der Bestimmung auf die Einordnung der Situationen, bei welchen Schwarz am Zug ist geachtet werden.
Da beim späteren Verwenden der KI die Züge des schwarzen Spielers nicht beeinflusst werden können, muss jeder mögliche
Zug einer Situation in $S_{n+1}$ mit $n \% 2 = 0$ zu einer Situation aus $S_n$ führen. Diese Überprüfung wird mit der Funktion
`check_black_determinism` durchgeführt.
Da die Züge von Weiß gezielt gewählt werden können, ist diese Überprüfung bei $n \% 2 = 1$ nicht nötig. 

Außerdem müssen bei der Durchführung des Algorithmus weitere Aspekte berücksichtigt werden:
* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden. Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen, können zu einer anderen Figur eingetauscht werden. Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten Reihe befinden, diese manuell durch einen Bauern ersetzt.
 
Die Umsetzung erfolgt durch die Funktion ``previous_states``. 
Alle Funktionsparameter können aus der nachfolgenden Liste entnommen werden:

* ``used_boards``: Die Menge aller bereits einem $n$ zugeordneten Situationen, welche nicht nocheinmal beachtet werden sollen.
* ``iteration_count``: $m$ des $S_m$, welches gerade berechnet wird.
* ``user_wants_pawn``: Ein Flag, welches steuert, ob spezifische Bewegungen des Bauern berechnet werden sollen.
* ``uniques``: Die Situationen, welche als Urpsrung der Spiegelung verwendet werden.

Der Algorithmus zur Bestimmung der Menge $S_{n+1}$ wird im folgenden Abschnitt beschrieben.

* Über die Uniques (ungespiegelte Situationen) iterieren.
  * Den Spieler, welcher am Zug ist, wechseln (Da, um im aktuellen Zustand anzukommen, der andere Spieler einen Zug
  gemacht hat)
  * Alle Positionen mit Bauern berechnen 
  * Alle pseudo-legalen Bewegungen mittels der funktion `regular_moves` durchführen. 
    Hierbei werden keine Züge der Bauern beachtet. Die technische Umsetzung wird in der Dokumentation der Funktion erklärt.
  * Wenn die Bewegungen von Bauern abgebildet werden müssen:
    * Bauern manuell einen Schritt "nach hinten" setzen.
    * Überprüfen, ob eine Dame in der obersten Reihe durch einen Bauern in der vorletzten ersetzt werden muss.
    * Die technische Umsetzung dieser Aktionen wird in der Dokumentation der Funktionen `pawn_moves` und 
      `replace_queen_with_pawn` erklärt.
  * Den Spieler, welcher ursprünglich am Zug war, wiederherstellen.
  * Wenn $(n+1) \% 2 = 0$ überprüfen, ob alle zuvor berechneten Boards mit allen Moves in $S_n$ enden. 

Außerdem müssen bei der Durchführung des Algorithmus weitere Aspekte berücksichtigt werden:

* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden. Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen, können zu einer anderen Figur eingetauscht werden. Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten Reihe befinden, diese manuell durch einen Bauern ersetzt.

Die Funktion bestimmt die Menge $S_{n+1}$, die Menge der bekannten Boards als Tupel `used_boards` und die ungespiegelten Origniale 
aus $n+1$ `s_n1_uniques`

In [ ]:
def previous_states(used_boards, iteration_count, user_wants_pawn, uniques):
    #variables
    s_n1 = []
    s_n1_tuples = set()
    s_n1_uniques = []
    s_n1_uniques_tuples = set()

    for i in range(len(uniques)):
        status = "Calculating S" + str(iteration_count) + " - Board " + str(i+1) + " of " + str(len(uniques)) + " from S" + str(iteration_count-1)
        clear_output(wait=True)
        print(status)

        # Copy current board and invert the player
        chess_board = uniques[i].copy()
        chess_board.turn = chess_board.turn ^ True

        # Find all Pawns
        pawn_positions = find_pawns(chess_board)
        
        # try moves and check if they lead to new boards
        tmp_n1, tmp_n1_tuples, tmp_uniques, tmp_uniques_tuples = regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions, user_wants_pawn)
        s_n1 += tmp_n1
        s_n1_tuples |= tmp_n1_tuples
        s_n1_uniques += tmp_uniques
        s_n1_uniques_tuples |= tmp_uniques_tuples

        if user_wants_pawn and chess_board.turn:
            # Push all pawns one row back and check if this leads to new boards
            if len(pawn_positions) > 0:
                tmp_list, tmp_set = pawn_moves(chess_board, used_boards, s_n1_tuples)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set
                s_n1_uniques += tmp_list
                s_n1_uniques_tuples |= tmp_set
            
            # Exchange Queens with Pawns
            queen_positions = check_top_row_for_queen(chess_board)
            if queen_positions:
                tmp_list, tmp_set = replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, queen_positions)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set
                s_n1_uniques += tmp_list
                s_n1_uniques_tuples |= tmp_set
            

        # Restore the original state of the board
        chess_board.turn = chess_board.turn ^ True

    # Only needed for Black-Moves
    if iteration_count % 2 == 0:
        clear_output(wait=True)
        print("Calculating S" + str(iteration_count) + " - Checking Black Moves for determinism")
        s_n1, s_n1_tuples, s_n1_uniques = check_black_determinism(s_n1, used_boards, s_n1_uniques_tuples)

    clear_output(wait=True)
    print("Done with S" + str(iteration_count))
    return s_n1, used_boards | s_n1_tuples, s_n1_uniques

## Hilfsfunktionen für die Berechnung

Die folgenden Funktionen werden zur Berechnung der previous_states verwendet. 
Sie übernehmen dabei diverse Aufgaben wie das Durchführen von regulären Moves oder das "manuelle" Versetzen von Figuren, um eine andere Situation zu generieren. 


Die Funktion ``regular_moves`` führt für eine übergebene Situation `chess_board` alle `pseudo_legal_moves` durch, um mögliche vorhergehende Situationen zu berechnen.  
Pseudo-Legale-Züge sind Züge, welche die Figuren auf eine Art bewegen, die der Figur gestattet ist, aber unter Umständen in eine nicht legale Spielsituation führt. 
Diese werden verwendet, da nur weil der Move von $S_{n+1}$ zu $S_n$ legal ist, der Zug umgekehrt dies nicht sein muss.

Ein simples Beispiel:
Situation $S_n$: Ein König befindet sich ein Feld von einem Schach entfernt.  
Diese Position kann erreicht worden sein, da der König von einer Position in $S_{n+1}$ sich aus diesem Schach herausbewegt hat.
Der Zug "in das Schach", wäre jedoch nicht legal, weshalb ein Move aus der Liste der `pseudo_legal_moves` zur Berechnung genommen werden muss.
Dies funktioniert nicht für Bauern, da ein Schritt nach "hinten" keine Bewegung ist, welche der Figur zusteht.

Die Funktion überprüft jeden Zug, welcher in der Situation möglich ist. Wenn die errechnete Situation valide und noch nicht verwendet (überprüft durch Einträge in `used_boards` und `s_n1_tuples`) ist, wird sie den Rückgabe-Variablen angefügt.
Wenn sich keine Bauern auf dem Spielfeld befinden (`user_wants_pawn`), dann können die Situationen gespiegelt werden, um weiteren Rechenaufwand zu reduzieren.
Diese Spiegelung findet durch eine Iteration über die später definierten Swap_Types statt. Anschließend wird mit der Funktion `mirror` die Spiegelung bestimmt,
die Validität der Situation überprüft und ebenfalls an das Ergebnis angefügt.

In [ ]:
def regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions, user_wants_pawn):
    new_boards = []
    new_tuples = set()
    new_uniques = []
    new_uniques_tuples = set()
    for pLMove in chess_board.pseudo_legal_moves:
        if chess.square_name(pLMove.from_square) not in pawn_positions:
            chess_board.push(pLMove)
            chess_board.turn = chess_board.turn ^ True
            if not chess_board.is_valid():
                chess_board.turn = chess_board.turn ^ True
                chess_board.pop()
                continue
            # If the new board is found in S, it can be reached in one step
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples and tuple_rep not in new_tuples:
                new_uniques.append(chess_board.copy())
                new_uniques_tuples.add(tuple_rep)
                
                new_boards.append(chess_board.copy())
                new_tuples.add(tuple_rep)
                
                if not user_wants_pawn:
                    for swtype in Swap_Type:
                        mirrored_board = mirror(chess_board, swtype)
                        tuple_rep_mir = (mirrored_board.turn,mirrored_board.__str__())
                        if tuple_rep_mir not in used_boards and tuple_rep_mir not in s_n1_tuples and tuple_rep_mir not in new_tuples:
                            new_boards.append(mirrored_board.copy())
                            new_tuples.add(tuple_rep_mir)
            chess_board.turn = chess_board.turn ^ True
            chess_board.pop()
    
    return new_boards, new_tuples, new_uniques, new_uniques_tuples


Wie zuvor bereits erwähnt, ermöglicht die quadratische Natur des Schachbrettes es das Spielbrett zu spiegeln / rotieren und weitere Situationen zu erhalten. 

Zunächst wird ein Enum erstellt, welches es ermöglicht über die Arten der Figurenvertauschungen zu iterieren.
`Swap_Type` übersetzt zu einem String, welcher im nächsten Schritt als Key für ein Dictionary verwendet wird.

In [ ]:
class Swap_Type(Enum):
    VERTICAL = "vertical"
    HORIZONTAL = "horizontal"
    ROTATE_RIGHT = "rotate_right"
    ROTATE_180 = "rotate_180"
    ROTATE_LEFT = "rotate_left"

Für den Tausch wird über jede Figur iteriert und diese an die entsprechende Position gesetzt.
Das Ergebnis wird als `Board-Objekt` zurückgegeben.

In [ ]:
def mirror(board, sw_type : Swap_Type):
    swaps = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)}
    }
    
    swapped_board = chess.Board()
    swapped_board.clear()
    swapped_board.turn = board.turn

    for position, piece in board.piece_map().items():
        swapped_board.set_piece_at(swaps[sw_type.value][position], piece)
    
    return swapped_board

Befinden sich Bauern in der Situation, müssen diese manuell platziert werden, da für diese auch in den `pseudo_legal_moves` nur die Züge $S_n \rightarrow S_{(n+1)}$ aufgeführt sind.
Die Funktion `pawn_moves` erfüllt diese Anforderung.
Ähnlich wie die Funktion `regular_moves` werden für eine Situation `chess_b oard` alle Situationen berechnet, welche durch Bewegung eines Bauerns zu `chess_board` werden. 
Hierfür wird über alle Bauern auf dem Spielfeld iteriert, diese Entfernt und auf das Feld mit dem Index $n-9$ wieder gesetzt. Da eine Reihe
8 Felder hat, hat das Feld in derselben Linie aber vorherigen Reihe den Index 9 geringer. 
Auch diese Situationen werden sowohl auf Validität als auch bisheriges Vorkommen überprüft, bevor sie den Rückgabevariablen angefügt werden.
Das übergebene Objekt wird zu seinem Ursprungszustand zurückgeführt.

In [ ]:
def pawn_moves(chess_board, used_boards, s_n1_tuples):
    new_boards = []
    new_tuples = set()
    chess_board = chess_board.copy()
    chess_board.turn = chess_board.turn ^ True
    
    for pawn in chess_board.pieces(chess.PAWN, True):
        if chess_board.piece_at(pawn - 8) is None:
            chess_board.remove_piece_at(pawn)
            chess_board.set_piece_at(pawn - 8, chess.Piece.from_symbol('P'))
            if chess_board.is_valid():
                tuple_rep = (chess_board.turn,chess_board.__str__())
                if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                    new_boards.append(chess_board.copy())
                    new_tuples.add(tuple_rep)
            
    return new_boards, new_tuples


Ein Problem, das bei der Verwendung der Rückwärts-Analyse auftritt, liegt in dem Szenario: 
"König und Bauer gegen König". Dieses Szenario beinhaltet die Umwandlung des Bauerns, welcher die oberste Zeile erreicht hat, in eine andere Figur (Dame, Turm, Läufer, Springer). 
Da die Dame die stärkste Figur im Spiel ist, wird immer dieser Tausch gewählt.
Hat der Nutzer bei den weißen Figuren, welche sich in der Situation sollen, einen Bauern angegeben, wurde dieser beim Errechnen der Menge $S_0$ durch eine Königin ersetzt.

Für die Berechnung der idealen Züge muss der Bauer wieder in die Situationen, welche sich in den $S_n$ Mengen befinden, eingeführt werden.
Der Tausch eines Bauerns zu einer Dame kann nicht durch die `pseudo_legal_moves` umgekehrt werden.

Die Funktion ``check_top_row_for_queen`` überprüft, ob ein solcher Tausch möglich ist. Sie erhält als Parameter eine Situation `board`, für welches die Felder der obersten Zeile überprüft und jedes zurückgegeben wird, auf dem sich eine Dame befindet.

In [ ]:
def check_top_row_for_queen(board):
    return_list = []
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return_list.append(i)

    if len(return_list) > 0:
        return return_list
    else:
        return False

Wurden mittels der vorhergehenden Funktion Damen in der obersten Zeile gefunden, ersetzt `replace_queen_with_pawn` alle diese Positionen (`toprow_queen_positions`) durch einen 
Bauern in der vorletzten Zeile.
Es wird über die übergebenen Positionen von Damen in der obersten Reihe iteriert, diese entfernt und in der Reihe davor (Feld Index um 9 verringert) ein Bauer platziert.
Wenn die Situation ein valides Schachbrett darstellt, wird sie an die Rückgabeliste angefügt.

In [ ]:
def replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, toprow_queen_positions):
    new_boards = []
    new_tuples = set()
    
    for square in toprow_queen_positions:
        chess_board.remove_piece_at(square)
        chess_board.set_piece_at(square - 8, chess.Piece.from_symbol('P'))
        if chess_board.is_valid():
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                new_boards.append(chess_board.copy())
                new_tuples.add(tuple_rep)
        chess_board.remove_piece_at(square - 8)
        chess_board.set_piece_at(square, chess.Piece.from_symbol('Q'))
    return new_boards, new_tuples

Da Bauern mittels der Funktion `pawn_moves` gesondert behandelt werden müssen, muss in `regular_moves` verhindert werden, dass Züge mit Bauern durchgeführt werden.
Hierfür wird die Information benötigt, auf welchen Feldern sich ein Bauer befindet. Diese Information wird durch die Funktion `find_pawns` generiert.  

In [ ]:
def find_pawns(chess_board):
    result = []
    for pawn in chess_board.pieces(chess.PAWN, True):
            result.append(chess.square_name(pawn))
    return result

Wenn mittels der Ki eine Spielsituation ausgewertet wird, kann für jeden Zug des weißen Spielers ein Zug ausgewählt werden.
Für die Situationen, bei denen Schwarz am Zug ist, muss die Ki alle möglichen Züge auswerten können.
Da jedoch für einen spezifischen Zug, welcher eine Situation von $S_n$ in $S_{n-1}$ führt, dasselbe nicht für alle Züge, welche in der Situation möglich sind gilt,
müssen die Situationen, bei welchen Schwarz am Zug ist, besonders gefiltert werden.
Für jede Situation $b$ aus einem $S_n$ mit $n \% 2 = 0$ muss folglich gelten:  
$$
b \in S_n \implies \forall m \in valid\_moves(b): b.push(m) \in S_{m} \land m < n
$$
Wobei `valid_moves` die Liste der legalen Züge für eine Situation ist und `b.push(m)` die Situation beschreibt, welche durch Ausführen des Zuges $m$ entsteht. 

Die Funktion `check_black_determinism` stellt dies sicher.
Für jede Situation in `s_n1` wird jeder mögliche legale Zug ausgeführt und überprüft, ob die entstehende Situation in einer Menge
$S_m$ mit $m <= n$ auffindbar ist. Nur wenn alle Züge diese Bedingung erfüllen, wird das Objekt in die Liste `s_n1_tmp`, welche in der
funktion `previous_states` die eigentliche Liste `s_n1` ersetzen wird, auf genommen.

In [ ]:
def check_black_determinism(s_n1, used_boards, uniques_n1_tuples):
    s_n1_tmp = []
    s_n1_tuples_tmp = set()
    uniques_n1_tmp = []
    
    
    for chess_board in s_n1:
        include = True
        if not chess_board.turn:
            for move in chess_board.legal_moves:
                chess_board.push(move)
                tuple_rep = (chess_board.turn,chess_board.__str__())
                chess_board.pop()
                if tuple_rep not in used_boards:
                    include = False
        if include:
            s_n1_tmp.append(chess_board)
            s_n1_tuples_tmp.add((chess_board.turn,chess_board.__str__()))
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep in uniques_n1_tuples:    
                uniques_n1_tmp.append(chess_board)
    
    return s_n1_tmp, s_n1_tuples_tmp, uniques_n1_tmp

## Export in Datei
Nach erfolgreicher Berechnung einer $S_n$ Menge werden die FENs der Situationen in eine temporäre `.preConvert` Datei geschrieben.
Wurden alle $S_n$ berechnet, wird die temporäre Datei in eine `.chessAI` Datei für die Verwendung in der Ki und eine `.chessTest` für das 
Testen der Ergebnisse konvertiert.

Damit keine Werte einer vergangenen Berechnung in der temporären Datei vorliegen, muss zuerst eine leere `.preConvert` Datei erstellt werden. 

In [ ]:
def create_empty_file(filename):
    f = open("S_n_Results/" + filename + ".preConvert", "w")
    f.write("")
    f.close()

Die Zwischenergebnisse werden stetig mittels der `append_to_file` Funktion an die zuvor erstellte `.preConvert` Datei angehängt.
Jede Zeile entspricht hierbei einem $n$ aus den $S_n$ Mengen.
Für die Zwischenergebnisse werden die FENs als JSON gespeichert.

In [ ]:
def append_to_file(s_n, filename):
    s_n_ascii = []
    for board in s_n:
        s_n_ascii.append(board.fen())

    f = open("S_n_Results/" + filename + ".preConvert", "a")
    f.write("\n")
    f.write(json.dumps(s_n_ascii))
    f.close()

Nachdem die gesamte Sequenz aller $n$ berechnet wurde, muss die temporäre Datei in zwei Dateien zur Auswertung konvertiert werden.
Die `.chessAI` Datei enthält Mengen von Tupeln.
Die `.chessTest` Datei enthält die FENs, um in den Test-Szenarien wieder Board-Objekte erstellen zu können.

Zuerst müssen die Informationen über die berechneten $S_n$ Mengen aus der `.preConvert` Datei gelesen werden. Die darin gespeicherten FENs werden in einer Liste gespeichert
und zusätzlich zu Board-Objekten instanziiert. Die Objekte werden in die zuvor bereits verwendete Tupel-Darstellung gewandelt und in eine Menge eingefügt. 

Zum Speichern der Dateien wird aus Effizienzgründen das Modul Pickle verwendet, welches die Daten in Binärdateien speichert. 
Da die `.chessAI` Dateien gegebenenfalls an Nutzer der KI verteilt werden müssen, werden diese zusätzlich mit dem Modul `ZipFile` komprimiert. Die `.chessTest` Dateien werden
nur zum Evaluieren der Ergebnisse verwendet und nicht an Nutzer verteilt. Damit sie schneller eingelesen werden können, werden sie nicht komprimiert.

 

In [ ]:
def convert_file(filename):
    s_n_seq_fens = []
    s_n_seq_tuples = []
    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp_set = set()
        tmp = json.loads(line)

        for fen in tmp:
            tmp_list.append(fen)
            tmp_board = chess.Board(fen)
            tmp_set.add((tmp_board.turn,tmp_board.__str__()))
        s_n_seq_fens.append(tmp_list)
        s_n_seq_tuples.append(tmp_set)
    f.close()

    f = open("S_n_Results/" + filename + ".pickle", "wb")
    f.write(pickle.dumps(s_n_seq_tuples))
    f.close()
    
    f = open("S_n_Results/" + filename + ".chessTest", "wb")
    f.write(pickle.dumps(s_n_seq_fens))
    f.close()
    
    with ZipFile("S_n_Results/" + filename + '.chessAI', 'w', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + filename + ".pickle", filename+".pickle")
    if os.path.exists("S_n_Results/" + filename + ".chessAI") and os.path.exists("S_n_Results/" + filename + ".pickle"):
        os.remove("S_n_Results/" + filename + ".pickle") 

    if os.path.exists("S_n_Results/" + filename + ".preConvert"):
        os.remove("S_n_Results/" + filename + ".preConvert")

Mit den zuvor definierten Funktionen ist es nun möglich, alle $S_n$ zu bestimmen. Hierfür wird die Funktion ``previous_states`` solange aufgerufen, bis keine Situationen
für ein $n+1$ mehr gefunden werden und die Liste $S_{n+1}$ leer ist.

Die Funktion `run` geht hierbei von einem `s_n`, welches zuvor bestimmt wurde, aus und durchläuft diesen Prozess in einer Schleife.
Nach abgeschlossener Rechnung wird das Konvertieren der Datei gestartet.

In [ ]:
def run(s_n, used_boards, n, user_wants_pawn, filename, uniques):
    if n == 0:
        append_to_file(s_n, filename)

    while True:
        n += 1
        s_n, used_boards, uniques = previous_states(used_boards, n, user_wants_pawn, uniques)
        print("S " + str(n) + ": " + str(len(s_n)))
        if not s_n: #an empty list is false
           break
        append_to_file(s_n, filename)
        del s_n

    print("Done")
    print(str(n) + " S-Lists calculated")

    convert_file(filename)

    print("File converted")

Die Funktion `run_from_start` berechnet $S_0$ und startet anschließend die Berechnung aller $S_n$

In [ ]:
def run_from_start(user_supplied_pieces, filename):
    user_wants_pawn = chess.Piece.from_symbol("P") in user_supplied_pieces
    
    s_0, used_boards, uniques = setup_boards(user_supplied_pieces, user_wants_pawn)
    print(len(used_boards))
    print(len(uniques))
    create_empty_file(filename)
    run(s_0, used_boards, 0, user_wants_pawn, filename, uniques)

Wurde die Berechnung abgebrochen (z.B. durch einen Neustart des Computers) kann die Funktion `resume_from_file` den Zustand, welcher in einer `.preConvert` Datei gespeichert wurde
wiederherstellen, und die Berechnung beim nächsten $n$ fortsetzen.
Hierfür werden alle bereits berechneten $n$ in Objekte instanziiert und die Tupel Repräsentationen als bereits verwendete Situationen (`used_boards`) gespeichert.
Für das höchste $n$, werden die Objekte behalten und der `run` Funktion übergeben.

In [ ]:
def resume_from_file(filename):
    used_boards = set()
    s_n = []
    count = -1

    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp_set = set()
        tmp = json.loads(line)

        for fen in tmp:
            chess_board = chess.Board(fen)
            tmp_list.append(chess_board)
            tmp_set.add((chess_board.turn,chess_board.__str__()))
        s_n = tmp_list
        used_boards |= tmp_set
        count += 1
    f.close()
    print("Starting at S" + str(count) + " (" + str(len(s_n)) + " Boards)")
    
    # Adding a few Pawns to the result only slightly increases the file_size, and the value of user_wants_pawn is \
    # currently not stored
    run(s_n, used_boards, count, True, filename, s_n)

## Konfigurations Variablen

Für die Bestimmung der gewonnenen Spielbretter müssen nun die Spielfiguren angegeben werden, für die die Endspiel-Datenbank berechnet werden soll. Hierfür werden in der Liste ``pieces_to_place`` alle Figuren aufgeführt.
Die Einträge der Liste werden als Tupel bestehend aus Figur und Farbe gespeichert.
Bsp.: ``(chess.KING, chess.WHITE)``
Die Reihenfolge oder Position der Figur ist für die Berechnung irrelevant.
Diese wird erst im nächsten Schritt (der Auswertung) benötigt.

Der ``Filename`` wird für das Speichern der Ergebnisse verwendet.

In [ ]:
WHITE_PIECES_TO_PLACE = [chess.Piece.from_symbol('P')]

FILENAME = "S_n_seq_pawn_15_03"

## Start der Rechnung

In [ ]:
%%time
run_from_start(WHITE_PIECES_TO_PLACE, FILENAME)
#resume_from_file(FILENAME)